# Simple PageRank implementation with SparkRDD

In [ ]:
def parse_neighbours(urls):
    """
    Parse an URL pairs string into URL-s tuple
    """
    parts = urls.split(r"\s+", urls, 1)
    return parts[0], parts[1]


def compute_contribs(urls, rank):
    """
    Calculates URL contributions to the rank of other URL-s
    """
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


# Load input data
# The data should be in format:
#   URL   neighbour_URL
#   URL   neighbour_URL
#   ...
raw = spark_session.read.text(sys.argv[1]).rdd.map(lambda x: x[0])

links = raw.map(lambda x: parse_neighbours(x)) \
           .distinct() \
           .partitionBy(100) \
           .cache()

ranks = links.mapValues(lambda x: (x[0], 1.0))

for i in range(int(sys.argv[2])):
    contribs = links.join(ranks).flatMap(lambda x: compute_contribs(x[1][0], x[1][1]))
    ranks = contribs.reduceByKey(add).mapValues(lambda x: x * 0.85 + 0.15)
    
for (link, rank) in ranks.collect():
    print(link, rank, sep="\t")